# Functional analysis CIN-subtypes (Micoli et al.)

#### Overview

This notebook demonstrates the analysis pipeline described in **Micoli et al., 2025**: *Decoding the Genomic and Functional Landscape of Emerging Subtypes in Ovarian Cancer.*

#### Contents

* Data loading
* Cell type analysis
* Differential expression analysis (DEA)
* Transcription factor activity inference (via CollecTRI)
* Pathway activity inference (via PROGENy)
* Gene Set Enrichment Analysis (GSEA)

#### Required Inputs

* PRISM<sup>1</sup> output: Required for downstream inference analyses.
* Expression matrix: Gene expression counts with genes labeled by their gene symbols (EOC component from PRISM)
* Annotation table: Must include grouping variables for comparison and any covariates for DEA.

#### Use Case

This notebook is designed for comparisons:

* Between one subtype and the others within a single tissue type (either metastatic or site-of-origin)
* Or between two tissue types within the same subtype

Ensure the appropriate expression matrix and annotation table are provided for each analysis.

<br><br>
<sup>1</sup> Häkkinen, Antti et al. “PRISM: recovering cell-type-specific expression profiles from individual composite RNA-seq samples.” Bioinformatics (Oxford, England) vol. 37,18 (2021): 2882-2888. doi:10.1093/bioinformatics/btab178

In [ ]:
# Loading of packages
import scanpy as sc
import decoupler as dc

# Only needed for processing
import numpy as np
import pandas as pd
from anndata import AnnData
import conorm
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu, ttest_ind, levene, shapiro

# Import DESeq2
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats

# Import GSEApy
import gseapy as gp

In [ ]:
sns.reset_defaults()
plt.rcParams['figure.facecolor'] = (1,1,1,1)
plt.rcParams['figure.dpi'] = 800
sns.set_style('white')

## Data loading

In [ ]:
# Load annotation and expression matrix
ann_samples = pd.read_csv("path/to/annotations.tsv", sep="\t", index_col = 0)
ann_samples = ann_samples.set_index("rna_sample") # Move column to rownames

E = pd.read_csv(counts_E1_path, sep='\t', index_col=0)
E = E.set_index("X")

Example of annotation table:

| Sample ID | Comparison  | Tissue    | 
|-----------|-------------|-----------|
| S001      | EMT         | Ovary     |
| S002      | other_HRP   | Tube      |
| S003      | other_HRP   | Adnexal   |
| S004      | EMT         | Tube      |
| ...       | ...         | ...       |

### Normalization of counts
TMM normalization fro the GSEA analysis

In [ ]:
E_tmm = conorm.tmm(E)

## Cell types imbalances 

Analysis of the main cell types aboundancies in the comparison groups (epithelial ovarian cancer cells, immune cells, fibroblasts and other cells).

Required: from PRISM load decomposition values (W matrix).

In [ ]:
# Load weigths
W = pd.read_csv('path/to/W.tsv.gz', sep='\t', index_col=0)

# Filter for samples in annotation 
W = W.loc[ann_samples.index]

# Add cluster and sample columns
W['Cluster'] = ann_samples.loc[W.index, 'comparison']
W.index.name = 'Sample'
W['sample'] = W.index

# Tranform in a wider format
Wm =pd.melt(W, id_vars=['sample', 'Cluster'], var_name='CellType_Abundance')
Wm = Wm[Wm.CellType_Abundance.isin(['EOC', 'Fibroblast', 'Immune', 'Unknown'])]

In [ ]:
# Plot differences
ax = sns.boxplot(data=Wmelted, x='CellType_Abundance', y='value', hue='Cluster',
                 palette = {"Group1": '#9b5de5', "Group2": '#95d5b2'},
                 hue_order=["Group1","Group2"] # Change this in favor of your categories
                )
ax.set_xticklabels(ax.get_xticklabels(), fontsize=10, rotation=-60)

ax.set_ylabel('% predicted by PRISM', fontsize=8)
ax.set_xlabel('', fontsize=0)
ax.set_title('Cell abundancies differences', fontsize=10)
plt.savefig('your/path/celltypes.png')

plt.show()

### Test for each celltype component if the classes are significantly different
For each celltype test first if the assumptions for the T test hold and then perform T test or Mann Whitney U-test.

In [ ]:
# Define the components to test
components = ['EOC', 'Fibroblast', 'Immune']

# Run the test for each component
for component in components:
    core_values = W[W.Cluster == 'Group1'][component].values
    other_values = W[W.Cluster == 'Group2'][component].values
    
    stat, p_value = mannwhitneyu(core_values, other_values)
    
    significance = (
        "NOT significant" if p_value > 0.05 
        else "slightly significant" if p_value > 0.01 
        else "significant"
    )
    
    print(f"{component}: Mann-Whitney U = {stat}, p-value = {p_value:.5f} -> {significance}")

## DEA

Step-by-step differential expression analysis.

Required: expression matrix, annotation table, PRISM weigths and gains (W and G tables).

In [ ]:
# Transform counts in anndata object
adata = AnnData(np.round(E_1[ann_samples.index].T), dtype=np.float32) # Use raw counts
adata.var_names_make_unique()

# Add annotation information
adata.obs['comparison'] = [ann_samples.loc[sample_id, 'comparison'] for sample_id in adata.obs.index]
adata.obs['tissue'] = [ann_samples.loc[sample_id, 'tissue'] for sample_id in adata.obs.index]
adata.obs['sample_id'] = adata.obs.index

### Quality control
Sanity check of amount of genes for which there is a minimal expression.

In [ ]:
# Obtain genes that pass the thresholds
genes = dc.filter_by_expr(adata, group='cluster',
                          min_count=10, min_total_count=15, large_n=1, min_prop=1)

len(genes)

In [ ]:
# Filter by the obtained genes
adata = adata[:, genes].copy()

### Differential expression analysis

In [ ]:
# Build the DESeq2 object
dds = DeseqDataSet(
    adata=adata,
    design_factors= 'comparison',
    refit_cooks=True,
    n_cpus=8, ref_level = ['comparison', 'Group1']
)

In [ ]:
# Compute LFCs
dds.deseq2()

In [ ]:
# Scale by PRISM EOC counts setting the size factors
G = pd.read_csv('your/path/G.tsv.gz',
                sep='\t', index_col=0)
W = pd.read_csv('your/path/W.tsv.gz',
                sep='\t', index_col=0)

nf_bulk = G.loc['bulk_gains']*W['EOC']
nf_bulk = nf_bulk / np.exp(np.mean(np.log(nf_bulk)))


nf_bulk_II = nf_bulk.loc[ann_samples['sample'].values]
nf_bulk_II.index = ann_samples[ann_samples['sample'].isin(nf_bulk_II.index)].index
dds.obsm['size_factors'] = nf_bulk_II.values

In [ ]:
# DEA
stat_res = DeseqStats(dds,
                      contrast=["comparison",'Group1','Group2'])
# Compute Wald test
stat_res.summary()

In [ ]:
# Extract the results
results_df = stat_res.results_df

# Extract differentially expressed genes (DEGs)
DEG  = results_df[results_df['padj'] <= 0.05] 

In [ ]:
# Plot DEGs in a volcano
dc.plot_volcano_df(results_df, x='log2FoldChange',
                   y='padj', top=25, dpi=250, lFCs_thr=1., sign_limit=6, sign_thr=0.05)
plt.title('DeSeq2 volcano plot')
plt.savefig('your/path/volcano.png')

plt.show()

In [ ]:
# Extract the result in a vector
mat = results_df[['stat']].T.rename(index={'stat': 'g1.vs.g2'})

## Transcriptional factor activity with CollecTRI
Functional analysis to infer the activities of trancription factors.

Required: results from DEA

In [ ]:
# Retrieve the libraries 
collectri = dc.get_collectri(organism='human', split_complexes=False)
progeny = dc.get_progeny(top=500)

In [ ]:
# Infer TF activities with ulm
tf_acts, tf_pvals = dc.run_ulm(mat=mat,
                               net=collectri,
                               verbose=True, min_n=10)

In [ ]:
# Plot the top20 TF activities
dc.plot_barplot(tf_acts, 'core.vs.others', top=20, vertical=True, figsize=(8,5)
                , dpi=250
               )
plt.title('CollecTRI TFs activities')
plt.savefig('your/path/collecTRI.png')

plt.show()

## Pathway activity inference with PROGENy
Functional analysis to infer pathway activity of pre-selected pathways.

Required: results from DEA

In [ ]:
# Infer pathway activities with mlm
pathway_acts, pathway_pvals = dc.run_mlm(mat=mat, net=progeny, verbose=True,)

In [ ]:
# Plot pathway activities
fig = dc.plot_barplot(pathway_acts, 'core.vs.others',
                top=25, vertical=False, dpi=200, figsize=(6,3),)
plt.title('PROGENy pathway activities', fontsize=12)
plt.xticks(ticks=range(14), fontsize=8)
plt.ylabel('Activity', fontsize=8)
plt.tight_layout()
plt.savefig('your/path/progeny.png', bbox_inches='tight', dpi=200)

plt.show()

## GSEA
Gene Set Enrichment Analysis using the Cancer Hallmark database from MSigDB.

Required: TMM normalized expression matrix, annotation table

In [ ]:
# Create the study object
tmm_study = E_tmm[ann_samples['sample'].values]
tmm_study.columns = ann_samples[ann_samples['sample'].isin(tmm_study.columns)].index

gs_res_hk = gp.gsea(data=tmm_study, 
                 gene_sets='MSigDB_Hallmark_2020', # or enrichr library names
                 cls=ann_samples.cluster,
                 permutation_type='gene_set',
                 permutation_num=1000, 
                 method='signal_to_noise',
                 threads=1, seed= 7, max_size=500)
gs_res_hk.res2d.sort_values('NES')

In [ ]:
# GSEA plot of specific pathways
terms = gs_res.res2d.Term

gp.gseaplot(rank_metric=gs_res.ranking,
         term=terms[1],
         **gs_res.results[terms[1]], figsize=(10,6),)
plt.show()